In [ ]:
!pip install optuna

In [ ]:
!pip install scikit-learn -q

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
files = []
y_list = []

for class_n, c in enumerate(['kiss', 'handshake', 'hug', 'highfive']):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    
    y_list.extend([class_n]*len(filtered))
    files.extend(filtered)

y = np.array(y_list)

x = [np.load(x)/255 for x in files]


In [ ]:
x.shape

In [ ]:
def split(X, y, test_size=0.05):
    shuffled_idx = np.random.permutation(len(X))
    
    split_at = int(len(shuffled_idx) * test_size)
    train_idx = shuffled_idx[split_at:]
    test_idx = shuffled_idx[:split_at]
    
    train_X = itemgetter(*train_idx)(X)
    train_y = y.take(train_idx, 0)
    
    test_X = itemgetter(*test_idx)(X)
    test_y = y.take(test_idx, 0)
    
    return (train_X, test_X, train_y, test_y)

In [ ]:
def onehot(y):
    onehot = np.zeros((y.size, 4))
    onehot[np.arange(y.size),y] = 1
    return onehot

In [ ]:
y = onehot(y)
train_X, test_X, train_y, test_y = split(x, y, 0.2)